In [1]:
#패키지 준비

import pymysql
import pandas as pd

In [2]:
movie_info_result = pd.read_csv('..\data-files\movie_info_result.csv', encoding='utf-8')

print(movie_info_result.info())
print(movie_info_result['directors_ko'][46916])
# *directors_ko, directors_en 안의 "[", "]", ",", "'", " " 모두 문자열로 인식되어 들어가 있는 문제

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46919 entries, 0 to 46918
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieCd       46919 non-null  object
 1   movieNm       46919 non-null  object
 2   movieNmEn     46919 non-null  object
 3   showTm        46110 non-null  object
 4   prdtYear      46824 non-null  object
 5   openDt        18630 non-null  object
 6   nationsNm     46919 non-null  object
 7   genreNm       46919 non-null  object
 8   directors_ko  46919 non-null  object
 9   directors_en  46919 non-null  object
 10  actors_ko     46919 non-null  object
 11  actors_en     46919 non-null  object
 12  cast          46919 non-null  object
 13  castEn        46919 non-null  object
 14  watchGradeNm  46919 non-null  object
dtypes: object(15)
memory usage: 5.4+ MB
None
['프랭크 후이', '구문걸', '비키 웅']


In [3]:
# 문제를 해결하기 위해 strip을 이용해 불필요한 특수문자('[',']',"'"")와 공백을 제거하고 ","을 기준으로 이름을 나눠주는 코드

test = movie_info_result
test['directors_ko'] = test['directors_ko'].str.strip("[]")
test['directors_ko2'] = test['directors_ko'].map(lambda x : x.split(","))

itsKo = []

for idx, item in enumerate(test['directors_ko2']):
    wow = []
    test['directors_ko2'][idx] = wow
    for it in item:
        it = it.strip(" ").strip("'")
        wow.append(it)
        itsKo.append(it)

In [4]:
# 잘 분리됐는지 구조를 확인하기 위한 코드
print(test['directors_ko2'])
print(test['directors_ko2'][46916])
print(test['directors_ko2'][46916][0])

0                 [콜린 트레보로우]
1                    [조 카나한]
2                      [오윤동]
3                  [알베르 뒤퐁텔]
4                      [김지곤]
                ...         
46914               [빈 분루에릿]
46915                  [엽위민]
46916    [프랭크 후이, 구문걸, 비키 웅]
46917              [산자이 가드비]
46918              [에밀리 데루제]
Name: directors_ko2, Length: 46919, dtype: object
['프랭크 후이', '구문걸', '비키 웅']
프랭크 후이


In [5]:
# movieCd와 복수의 peopleNm을 각각의 리스트로 나누는 작업 
# ex) [20171942, ['프랭크 후이','구문걸']] 
# --> [20171942, '프랭크 후이'], [20171942, '구문걸']

repo = []
director_matching = []
movie_order = 0

for movie in test['movieCd']:

    if movie == test.iloc[movie_order]['movieCd']:
        print(movie, test.iloc[movie_order]['movieCd'])

        for director in test.iloc[movie_order]['directors_ko2']:
            repo = [movie, director]
            director_matching.append(repo)
        movie_order += 1
        
    else:
        continue

20206061 20206061
20210547 20210547
20211413 20211413
20212027 20212027
20212064 20212064
20212063 20212063
20212042 20212042
20212062 20212062
20212041 20212041
20136895 20136895
20205986 20205986
20211176 20211176
20218415 20218415
20210611 20210611
20211241 20211241
20210031 20210031
20196264 20196264
20210864 20210864
20181118 20181118
20219056 20219056
20206464 20206464
20189638 20189638
20210689 20210689
20148592 20148592
20136792 20136792
20201815 20201815
20212015 20212015
20211929 20211929
20210663 20210663
20195959 20195959
20211159 20211159
20183772 20183772
20211220 20211220
20210790 20210790
20209295 20209295
20211735 20211735
20199882 20199882
19838068 19838068
20010264 20010264
20210882 20210882
20194403 20194403
20211734 20211734
20205736 20205736
20211768 20211768
20188441 20188441
20138950 20138950
20218390 20218390
20217102 20217102
20217167 20217167
20210794 20210794
20215801 20215801
20210028 20210028
20210883 20210883
20199962 20199962
20211994 20211994
20211978 2

In [6]:
# 위에서 만든 리스트를 [directorCd, peopleNm]과 매칭시키기 위해 데이터프레임 형식으로 전환

df = pd.DataFrame(director_matching, columns=['movieCd', 'peopleNm'])
df

,movieCd,peopleNm
0,20206061,콜린 트레보로우
1,20210547,조 카나한
2,20211413,오윤동
3,20212027,알베르 뒤퐁텔
4,20212064,김지곤
...,...,...
51594,20171942,프랭크 후이
51595,20171942,구문걸
51596,20171942,비키 웅
51597,20170306,산자이 가드비


In [7]:
# DB에서 directorCd, peopleNm 가져오는 작업

conn = pymysql.connect(host="localhost",
                       database="firstkino",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

all_directors = []
with conn.cursor() as cursor:
    cursor.execute("SELECT directorCd, peopleNm FROM Director")
    directors = cursor.fetchall()
    all_directors.append(directors)

conn.commit()

conn.close()

In [8]:
# directorCd, peopleNm이 튜플형식으로 되어있으므로 list형식으로 변환

director_list = list(all_directors[0])

director_df = pd.DataFrame(director_list, columns=['directorCd', 'peopleNm'])

print(director_df)

       directorCd       peopleNm
0               1               
1               2  세이드 M. 타바타바에이
2               3        프랑수아 롯제
3               4        조나단 거핀켈
4               5       레이 레이 아이
...           ...            ...
23461       23462            오정훈
23462       23463          리 동메이
23463       23464        가스통 듀프랫
23464       23465             소목
23465       23466       W. 스콧 피크

[23466 rows x 2 columns]


In [15]:
# df와 director_df를 peopleNm을 기준으로 병합

result = pd.merge(df, director_df, on='peopleNm')

In [16]:
# 전처리 중 생긴 필요없는 값 제거 및 제거 확인
print(result.loc[result['movieCd'] == 'movieCd'])

result = result.drop(result.loc[result['movieCd'] == 'movieCd'].index)

print(result.loc[result['movieCd'] == 'movieCd'])

       movieCd      peopleNm  directorCd
10954  movieCd  directors_ko       17402
Empty DataFrame
Columns: [movieCd, peopleNm, directorCd]
Index: []


In [17]:
# 중복값 찾기 및 삭제 코드
duplicateDFRow = result[result.duplicated()]
print(duplicateDFRow)
print("====================================================================")

result.drop_duplicates(keep = 'first', inplace = True)

duplicateDFRow = result[result.duplicated()]
print(duplicateDFRow)

        movieCd peopleNm  directorCd
57     20175865      정가영       14238
367    20198201      조은지       14634
7847   20195963      백승환       18902
7917   20179263      조성규       23288
8354   20200861      조혜린        8399
9193   20175182      신윤호        5270
16909  20135522       김현        1755
29337  20198183      김현호        2430
Empty DataFrame
Columns: [movieCd, peopleNm, directorCd]
Index: []


In [28]:
# DB에 저장하기 위해 list형식으로 변환

result_list = result.values.tolist()

In [33]:
# list형식의 데이터를 테이블에 넣는 작업
# *그냥하다가 중복값이 발생해서 위에서 중복값을 찾아 삭제해주는 작업을 수행

conn = pymysql.connect(host="localhost",
                       database="firstkino",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

with conn.cursor() as cursor:
    for post in result_list:
        print(post[0])
        cursor.execute("INSERT INTO MovieDirector (movieCd, directorCd) values (%s, %s)", (post[0], post[2]))
        print(post[1])
    conn.commit()

conn.close()

20206061
콜린 트레보로우
20098169
콜린 트레보로우
20129095
콜린 트레보로우
20210547
조 카나한
20216051
조 카나한
20030080
조 카나한
20120361
조 카나한
20100295
조 카나한
20070043
조 카나한
20124021
조 카나한
20211413
오윤동
20218815
오윤동
20212027
알베르 뒤퐁텔
20149696
알베르 뒤퐁텔
20181562
알베르 뒤퐁텔
20212064
김지곤
20212041
김지곤
20211682
김지곤
20181091
김지곤
20202138
김지곤
20148620
김지곤
20162761
김지곤
20150455
김지곤
20147090
김지곤
20158576
김지곤
20137561
김지곤
20126528
김지곤
20122134
김지곤
20212063
오민욱
20212042
오민욱
20196761
오민욱
20136610
오민욱
20159307
오민욱
20150472
오민욱
20137566
오민욱
20147174
오민욱
20140732
오민욱
20150460
오민욱
20198703
오민욱
20212062
손호목
20136895
김영조
20189666
김영조
20150457
김영조
20133805
김영조
20081657
김영조
20081751
김영조
20112633
김영조
20155483
김영조
20205986
정가영
20191042
정가영
20177483
정가영
20179495
정가영
20164142
정가영
20154968
정가영
20174303
정가영
20177981
정가영
20175865
정가영
20176582
정가영
20161806
정가영
20211176
필리프 팔라도
20134602
필리프 팔라도
20157421
필리프 팔라도
20061070
필리프 팔라도
20197988
필리프 팔라도
20218415
하마구치 류스케
20169305
하마구치 류스케
20210962
하마구치 류스케
20162491
하마구치 류스케
20162490
하마구치 류스케
20101347
하마구치 류스케